In [1]:
from coding_agents import run_to_clipboard


In [2]:
a = 'write a function that extract all the function from a python file to an output file named functions.py' 
b = 'write a function that get a string an execute it in the running environment '
a = 'write a function that get a py file and extract all the functions as a dictionary with: original function, function header, return and arguments'
run_to_clipboard(a)

'\nimport ast\nimport re\n\ndef extract_functions(file_path):\n    try:\n        # Read the file content\n        with open(file_path, \'r\') as file:\n            content = file.read()\n            \n        # Parse the Python code into an AST\n        tree = ast.parse(content)\n        \n        functions = {}\n        \n        # Walk through all nodes in AST\n        for node in ast.walk(tree):\n            if isinstance(node, ast.FunctionDef):\n                # Get the original function text\n                func_lines = content.split(\'\\n\')[node.lineno-1:node.end_lineno]\n                original = \'\\n\'.join(func_lines)\n                \n                # Get function header\n                header = func_lines[0]\n                \n                # Get arguments\n                args = []\n                for arg in node.args.args:\n                    args.append(arg.arg)\n                    \n                # Get return statements\n                returns = []\n     

In [9]:
print(extract_functions_to_dict('coding_agents.py'))

{'remove_non_python': {'original': 'def remove_non_python(text):\n    return text.replace("```python", "").replace("```", "")', 'header': 'def remove_non_python(text):', 'arguments': ['text'], 'returns': ['return text.replace("```python", "").replace("```", "")']}, 'generate_script': {'original': 'def generate_script(task_description, llm=llm, coding_agent_prompt=coding_agent_prompt):\n    prompt = coding_agent_prompt.format(task_description=task_description)\n    response = llm.complete(prompt)\n    return remove_non_python(response.text)', 'header': 'def generate_script(task_description, llm=llm, coding_agent_prompt=coding_agent_prompt):', 'arguments': ['task_description', 'llm', 'coding_agent_prompt'], 'returns': ['return remove_non_python(response.text)']}, 'fix_script': {'original': 'def fix_script(script, result, llm=llm, coding_agent_prompt=coding_agent_prompt):\n    prompt = coding_agent_prompt.format(task_description=f"The following script was generated to solve a task, but it

In [7]:

import ast
import re

def extract_functions_to_dict(file_path):
    try:
        # Read the file content
        with open(file_path, 'r') as file:
            content = file.read()
            
        # Parse the Python code into an AST
        tree = ast.parse(content)
        
        functions = {}
        
        # Walk through all nodes in AST
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                # Get the original function text
                func_lines = content.split('\n')[node.lineno-1:node.end_lineno]
                original = '\n'.join(func_lines)
                
                # Get function header
                header = func_lines[0]
                
                # Get arguments
                args = []
                for arg in node.args.args:
                    args.append(arg.arg)
                    
                # Get return statements
                returns = []
                for n in ast.walk(node):
                    if isinstance(n, ast.Return):
                        return_line = content.split('\n')[n.lineno-1].strip()
                        returns.append(return_line)
                
                functions[node.name] = {
                    'original': original,
                    'header': header,
                    'arguments': args,
                    'returns': returns
                }
                
        return functions
        
    except FileNotFoundError:
        print(f"Error: File {file_path} not found")
        return {}
    except SyntaxError:
        print(f"Error: Invalid Python syntax in {file_path}")
        return {}
    except Exception as e:
        print(f"Error: {str(e)}")
        return {}


In [ ]:

import re
import ast

def extract_functions(input_file, output_file='functions.py'):
    try:
        # Read input file
        with open(input_file, 'r') as f:
            code = f.read()
        
        # Parse code into AST
        tree = ast.parse(code)
        
        # Find all function definitions
        functions = []
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                # Get the source code for the function
                start = node.lineno - 1
                end = node.end_lineno
                func_lines = code.split('\n')[start:end]
                functions.append('\n'.join(func_lines))
        
        # Write functions to output file
        with open(output_file, 'w') as f:
            # Add imports from original file
            imports = []
            for node in ast.walk(tree):
                if isinstance(node, (ast.Import, ast.ImportFrom)):
                    import_lines = code.split('\n')[node.lineno-1:node.end_lineno]
                    imports.append('\n'.join(import_lines))
            
            if imports:
                f.write('\n'.join(imports) + '\n\n')
            
            # Write functions
            f.write('\n\n'.join(functions))
            
        return True
        
    except Exception as e:
        print(f"Error extracting functions: {str(e)}")
        return False

# Example usage:
# extract_functions('input.py')


In [ ]:

import difflib
import sys
from pathlib import Path

def compare_files(file1_path, file2_path):
    try:
        # Check if files exist
        if not Path(file1_path).exists() or not Path(file2_path).exists():
            raise FileNotFoundError("One or both files do not exist")
            
        # Read contents of both files
        with open(file1_path, 'r') as file1, open(file2_path, 'r') as file2:
            file1_lines = file1.readlines()
            file2_lines = file2.readlines()
            
        # Create differ object
        differ = difflib.Differ()
        
        # Get differences between files
        diff = list(differ.compare(file1_lines, file2_lines))
        
        # Print differences
        print("\nDifferences between files:")
        print("Lines starting with:")
        print("  '-' exist in first file but not second")
        print("  '+' exist in second file but not first") 
        print("  ' ' exist in both files\n")
        
        sys.stdout.writelines(diff)
        
        # Also return diff list for programmatic use
        return diff
        
    except Exception as e:
        print(f"Error comparing files: {str(e)}")
        return None

if __name__ == "__main__":
    # Example usage
    file1 = "main.ipynb"
    file2 = "bk5_main.ipynb"
    compare_files(file1, file2)



Differences between files:
Lines starting with:
  '-' exist in first file but not second
  '+' exist in second file but not first
  ' ' exist in both files

  {
   "cells": [
    {
     "cell_type": "code",
     "execution_count": 1,
     "metadata": {},
     "outputs": [
      {
       "data": {
        "text/plain": [
         "True"
        ]
       },
       "execution_count": 1,
       "metadata": {},
       "output_type": "execute_result"
      }
     ],
     "source": [
      "import sys\n",
      "import os\n",
      "from dotenv import load_dotenv\n",
      "import warnings\n",
      "import nest_asyncio\n",
      "import pyperclip\n",
      "nest_asyncio.apply()\n",
      "# warnings.filterwarnings('ignore')\n",
      "\n",
      "\n",
      "load_dotenv()\n"
     ]
    },
    {
     "cell_type": "code",
     "execution_count": 2,
     "metadata": {},
     "outputs": [],
     "source": [
      "from llama_index.llms.anthropic import Anthropic\n",
      "from llama_index.mult

In [ ]:
backup_revert_file('main.ipynb', 'backup')

Created backup: bk5_main.ipynb


True

In [ ]:

import os
import shutil
import re

def backup_revert_file(filename, operation):
    try:
        # Get directory and base filename
        directory = os.path.dirname(filename)
        base_name = os.path.basename(filename)
        
        if operation.lower() == 'backup':
            # Find existing backups to determine next number
            backup_pattern = f"bk[0-9]+_{base_name}"
            existing_backups = [f for f in os.listdir(directory or '.') 
                              if re.match(backup_pattern, f)]
            
            # Get next backup number
            if existing_backups:
                numbers = [int(re.search(r'bk(\d+)_', f).group(1)) 
                          for f in existing_backups]
                next_num = max(numbers) + 1
            else:
                next_num = 1
                
            # Create backup filename and copy file
            backup_name = f"bk{next_num}_{base_name}"
            backup_path = os.path.join(directory, backup_name) if directory else backup_name
            shutil.copy2(filename, backup_path)
            print(f"Created backup: {backup_name}")
            
        elif operation.lower() == 'revert':
            # Find all backups
            backup_pattern = f"bk[0-9]+_{base_name}"
            existing_backups = [f for f in os.listdir(directory or '.') 
                              if re.match(backup_pattern, f)]
            
            if not existing_backups:
                raise Exception("No backup files found")
                
            # Get latest backup
            numbers = [int(re.search(r'bk(\d+)_', f).group(1)) 
                      for f in existing_backups]
            latest_backup = f"bk{max(numbers)}_{base_name}"
            latest_path = os.path.join(directory, latest_backup) if directory else latest_backup
            
            # Replace current file with backup
            os.remove(filename)
            shutil.copy2(latest_path, filename)
            print(f"Reverted to backup: {latest_backup}")
            
        else:
            raise ValueError("Operation must be 'backup' or 'revert'")
            
    except Exception as e:
        print(f"Error: {str(e)}")
        return False
        
    return True


In [ ]:

import json
import re

def extract_functions_from_notebook(notebook_path, output_path='functions.py'):
    try:
        # Read the notebook file
        with open(notebook_path, 'r', encoding='utf-8') as f:
            notebook = json.load(f)

        # Open output file
        with open(output_path, 'w', encoding='utf-8') as out:
            # Write header comment
            out.write('# Functions extracted from {}\n\n'.format(notebook_path))
            
            # Go through each cell
            for cell in notebook['cells']:
                if cell['cell_type'] == 'code':
                    source = ''.join(cell['source'])
                    
                    # Find function definitions using regex
                    functions = re.finditer(r'(def\s+[^(]+\([^)]*\)\s*:(?:\s*[^\n]*\n+(?:\s+[^\n]*\n+)*)?)', source)
                    
                    # Write each function to output file
                    for func in functions:
                        func_text = func.group(1)
                        # Only write if it's a proper function definition
                        if func_text.strip():
                            out.write(func_text + '\n\n')
                            
        print(f"Functions have been extracted to {output_path}")
        
    except FileNotFoundError:
        print(f"Error: Could not find notebook file {notebook_path}")
    except json.JSONDecodeError:
        print(f"Error: Invalid notebook format in {notebook_path}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    # Example usage
    extract_functions_from_notebook('main.ipynb', 'functions.py')


Functions have been extracted to functions.py
